# 리스크 할당(Risk Budgeting) 모형 투자자 성향별 포트폴리오 산출

In [0]:
import numpy as np
import pandas as pd
import scipy.optimize as sco
import datetime as dt
from datetime import timedelta

# 예스트레이더 출력 결과물을 Pandas 형태로 포맷 변환

In [0]:
# 예스트레이더 종목검색 결과를 날짜 기준으로 포맷팅
def convert_yes_price_file(gubun):
    # 우리나라 시간 = 그리니치 표준시 + 9시
    file_dt = (dt.datetime.now() + dt.timedelta(hours=9)).strftime('%Y%m%d')

    yes_price_file = pd.DataFrame()
    # 예스트레이더 출력용 지표식과 칼럼수 일치시켜야 함.
    yes_price_file = pd.read_csv('yes_stock_price_' + file_dt + '.csv',encoding= 'euc-kr',
                           names = ['날짜1','종목명','종목코드','날짜','시가','고가','저가','종가','거래량',
                                    '단기이평','중기이평','장기이평','지수단기이평','지수중기이평','지수장기이평',
                                    'MACD','MACD_OSC','ATR','CCI','StoK','StoD','TRIX','이격도'],
                           index_col= 1, header=None, dtype={'종목코드':str})

    종목명 = yes_price_file.index.unique()
    종목코드 = yes_price_file['종목코드'].unique()  # 종목코드 사용할 경우를 위해 별도 저장
    df1 = {'종목명':종목명,
           '종목코드':종목코드}
    df1 = pd.DataFrame(df1)
    df1.to_csv('symbol_cd.csv')

    cls_p_data = pd.DataFrame()
    vol_data = pd.DataFrame()
    if gubun == 1:
        hi_p_data = pd.DataFrame()
        lo_p_data = pd.DataFrame()
        sma_data = pd.DataFrame()
        mma_data = pd.DataFrame()
        lma_data = pd.DataFrame()
        sema_data = pd.DataFrame()
        mema_data = pd.DataFrame()
        lema_data = pd.DataFrame()
        macd_data = pd.DataFrame()
        macdo_data = pd.DataFrame()
        atr_data = pd.DataFrame()
        cci_data = pd.DataFrame()
        stok_data = pd.DataFrame()
        stod_data = pd.DataFrame()
        trix_data = pd.DataFrame()
        dis_data = pd.DataFrame()

    for i in range(len(종목명)):
        cls_p = yes_price_file.loc[종목명[i],['날짜','종가']]
        cls_p.set_index(['날짜'],inplace=True)
        cls_p_data = pd.concat([cls_p_data,cls_p],axis=1) 

        vol = yes_price_file.loc[종목명[i],['날짜','거래량']]
        vol.set_index(['날짜'],inplace=True)
        vol_data = pd.concat([vol_data,vol],axis=1) 

        if gubun == 1:
            hi_p = yes_price_file.loc[종목명[i],['날짜','고가']]
            hi_p.set_index(['날짜'],inplace=True)
            hi_p_data = pd.concat([hi_p_data,hi_p],axis=1) 

            lo_p = yes_price_file.loc[종목명[i],['날짜','저가']]
            lo_p.set_index(['날짜'],inplace=True)
            lo_p_data = pd.concat([lo_p_data,lo_p],axis=1) 

            sma = yes_price_file.loc[종목명[i],['날짜','단기이평']]
            sma.set_index(['날짜'],inplace=True)
            sma_data = pd.concat([sma_data,sma],axis=1) 

            mma = yes_price_file.loc[종목명[i],['날짜','중기이평']]
            mma.set_index(['날짜'],inplace=True)
            mma_data = pd.concat([mma_data,mma],axis=1) 

            lma = yes_price_file.loc[종목명[i],['날짜','장기이평']]
            lma.set_index(['날짜'],inplace=True)
            lma_data = pd.concat([lma_data,lma],axis=1) 

            atr = yes_price_file.loc[종목명[i],['날짜','ATR']]
            atr.set_index(['날짜'],inplace=True)
            atr_data = pd.concat([atr_data,atr],axis=1) 

    cls_p_data.columns = 종목명
    print('\n 종가데이터 \n', cls_p_data.head())
    cls_p_data.to_csv('cls_p_data.csv')

    vol_data.columns = 종목명
    print('\n 거래량 \n', vol_data.head())
    vol_data.to_csv('volume_data.csv')

    if gubun == 1:
        hi_p_data.columns = 종목명
        print('\n 고가데이터 \n', hi_p_data.head())
        hi_p_data.to_csv('hi_p_data.csv')

        lo_p_data.columns = 종목명
        print('\n 저가데이터 \n', lo_p_data.head())
        lo_p_data.to_csv('lo_p_data.csv')

        sma_data.columns = 종목명
        print('\n 단기이평 \n', sma_data.head())
        sma_data.to_csv('sma_data.csv')

        mma_data.columns = 종목명
        print('\n 중기이평 \n', mma_data.head())
        mma_data.to_csv('mma_data.csv')

        lma_data.columns = 종목명
        print('\n 장기이평 \n', lma_data.head())
        lma_data.to_csv('lma_data.csv')

        atr_data.columns = 종목명
        print('\n ATR \n', atr_data.head())
        atr_data.to_csv('atr_data.csv')

In [0]:
# 채권ETF가격을 날짜 기준으로 포맷팅
def convert_yes_bond_etf_file():
    # 우리나라 시간 = 그리니치 표준시 + 9시
    file_dt = (dt.datetime.now() + dt.timedelta(hours=9)).strftime('%Y%m%d')

    yes_bond_etf_file = pd.DataFrame()
    yes_bond_etf_file = pd.read_csv('yes_etf_price_' + file_dt + '.csv',encoding= 'euc-kr',
                           names = ['날짜1','종목명','종목코드','날짜','시가','고가','저가','종가','거래량',
                                    '단기이평','중기이평','장기이평','지수단기이평','지수중기이평','지수장기이평',
                                    'MACD','MACD_OSC','ATR','CCI','StoK','StoD','TRIX','이격도'],
                           index_col= 1, header=None, dtype={'종목코드':str})

    종목명2 = yes_bond_etf_file.index.unique()
    종목코드2 = yes_bond_etf_file['종목코드'].unique()  # 종목코드 사용할 경우를 위해 별도 저장
    df2 = {'종목명':종목명2,
           '종목코드':종목코드2}
    df2 = pd.DataFrame(df2)
    df2.to_csv('symbol_cd2.csv')

    out_data2 = pd.DataFrame()
    for i in range(len(종목명2)):
        df2 = yes_bond_etf_file.loc[종목명2[i],['날짜','종가']]
        df2.set_index(['날짜'],inplace=True)
        out_data2 = pd.concat([out_data2,df2],axis=1) 
    out_data2.columns = 종목명2
    print(out_data2.head())
    out_data2.to_csv('bond_etf.csv')

In [0]:
gubun = 0 # 지표 파일을 만들려면 1로, 안 만들여면 0
convert_yes_price_file(gubun)
convert_yes_bond_etf_file()

# 사용자 함수 선언

In [0]:
def statistics(w,r,c):
    '''포트폴리오 총계치 출력
    인수
    =====
    w : array-like  포트폴리오 내의 비중
    
    반환값
    ======
    portfolio_return     : float 포트폴리오 수익률 기댓값
    portfolio_volatility : float 포트폴리오 변동성 기댓값
    sharpe_ratio         : float 무위험 이자율이 0일 때의 샤프 지수
    '''
    w = np.array(w)
    portfolio_return = np.sum(r * w)
    portfolio_volatility = np.sqrt(np.dot(w.T,np.dot(c, w)))
    sharp_ratio = portfolio_return / portfolio_volatility
    return np.array([portfolio_return, portfolio_volatility, sharp_ratio])

In [0]:
def risk_contribution(w,r,c):
    '''
    E(rp) = w'μ          : 포트폴리오 기대수익률
    σp orσ(w) = √(w'Ωw) : 포트폴리오 표준편차
    where w:= 구성종목 비중vector
          μ:= 자산별 기대수익률 vector
          Ω:= 공분산행렬
    Marginal Contribution to Risk of asset i(한계 리스크 기여도)
    MCRi = ∂σ(w)/∂wi = Δσ(w)/Δwi = 1/σ(w) * Σwi*cov(ri,rj) = Ωw/√(w'Ωw)
    := i번째 자산군 투자비중 변화에 대한 포트폴리오 전체 위험의 변화 정도
    Absolute Contribution to Risk of asset i(절대 리스크 기여도) 자산의 표준편차 값
    ACRi = wi*MCRi
    Percent Contribution to Risk of asset i(상대 리스크 기여도) 전체 중에 얼마인지 퍼센트로 표시한 것
    PCRi = ACRi/σ(w)
    '''
    sigma = statistics(w,r,c)[1]
    mcr = np.dot(c, w) / sigma
    acr = w * mcr
    pcr = acr / sigma
    return mcr, acr, pcr

In [0]:
# 최적포트폴리오의 투자비중과 포트폴리오 위험비중이 동일하게
def risk_budgeting_objective(w,r,c,front):
    sigma = statistics(w,r,c)[1]
    mcr = np.dot(c, w) / sigma
    return np.sum(np.square(front - (w * mcr / sigma)))

In [0]:
def min_func_sharpe(w,r,c):
# 최소화문제에서 샤프지수 최대값을 찾으려면 샤프지수의 음수 값을 최소화하면 된다.
    return -statistics(w,r,c)[2]  # 위에서 만든 statistics의 3번째 값이 샤프지수

In [0]:
def min_func_variance(w,r,c):
# 분산 최소화 함수
    return statistics(w,r,c)[1]**2

# Main

In [0]:
# 위험도별 주식비중
Wstk = (0.5, 0.7, 1.0)
공분산_산출기간 = 500  # 시뮬레이터를 통해 최적 값 찾을 것

# 주식종목 배분
raw_data = pd.read_csv('cls_p_data.csv',index_col=['날짜'], parse_dates=['날짜'])
종목명 = raw_data.columns
noa = len(종목명) # 자산수

symbol_data = pd.read_csv('symbol_cd.csv',index_col=None,dtype={'종목코드':str})
종목코드 = symbol_data['종목코드']
work_data = raw_data.iloc[-공분산_산출기간:]
현재가 = list(raw_data.iloc[-1])
print('\n >> 현재가 :', 현재가)

로그수익률 = np.log(work_data / work_data.shift(1))
print(로그수익률.head())

R = 로그수익률.mean() * 252 # 연수익률
print(R)

C = 로그수익률.cov() * 252  # 공분산, 표준편차가 아닌 분산이다
print(C)  

# 제약조건
cons = ({'type': 'eq', 'fun': lambda x: np.sum(x) - 1},
        {'type': 'ineq', 'fun': lambda x: (x >= 0.01).sum() - 4},   #매수 종목이 5 종목 이상되도록 조건 설정 
        {'type': 'ineq', 'fun': lambda x: (x >= 0.01).sum() - 3},
        {'type': 'ineq', 'fun': lambda x: (x >= 0.01).sum() - 2},
        {'type': 'ineq', 'fun': lambda x: (x >= 0.01).sum() - 1}) 

# 범위값
bnds = tuple((0, 1) for x in range(noa))

opts = sco.minimize(min_func_sharpe, noa * [1. / noa, ], (R, C), method='SLSQP', bounds=bnds, constraints=cons)
front = opts['x']  # 샤프지수 최대 포트폴리오 결과값

# 제약조건
cons2 = ({'type': 'eq', 'fun': lambda x: np.sum(x) - 1.0})

opts = sco.minimize(risk_budgeting_objective, noa * [1. / noa, ], (R, C, front), method='SLSQP', bounds=bnds,
                         constraints=cons2)
optsx = (opts['x'] * 100).round(3)  # 샤프지수 최대 포트폴리오 결과값
print('\n >> 비중 :', optsx)

rc_array = risk_contribution(opts['x'],R,C)
MRC = (rc_array[0] * 100).round(3)
ARC = (rc_array[1] * 100).round(3)
PRC = (rc_array[2] * 100).round(3)

stat = statistics(opts['x'],R,C).round(3)  # 샤프지수 최대 포트폴리오의 수익률, 표준편차, 샤프지수
print('\n>> 수익률 = ', stat[0],', 표준편차 = ', stat[1], ', 샤프지수 = ', stat[2])
print('\n>> 검증 >>')
print('mvo=', (front * 100).round(3))
print('PRC=', PRC.round(3))

# 우리나라 시간 = 그리니치 표준시 + 9시
current_dt = (dt.datetime.now() + timedelta(hours=9)).strftime('%Y%m%d')
date_col = [current_dt for i in range(noa)] # 날짜 칼럼 생성
print('\n >> 날짜 열 :', date_col)

mp11 = {'날짜':date_col,
      '종목명':종목명,
      '종목코드':종목코드,
      '비중':list((optsx * Wstk[0]).round(3)),
      '현재가':현재가,
      '한계리스크':MRC.round(3),
      '절대리스크':ARC.round(3),
      '상대리스크비중':PRC.round(3),
      'MVO':(front * 100).round(3)}
df11 = pd.DataFrame(mp11,columns = ['날짜','종목명','종목코드','비중','현재가','한계리스크','절대리스크','상대리스크비중','MVO'])
mp21 = {'날짜':date_col,
      '종목명':종목명,
      '종목코드':종목코드,
      '비중':list((optsx * Wstk[1]).round(3)),
      '현재가':현재가,
      '한계리스크':MRC.round(3),
      '절대리스크':ARC.round(3),
      '상대리스크비중':PRC.round(3),
      'MVO':(front * 100).round(3)}
df21 = pd.DataFrame(mp21,columns = ['날짜','종목명','종목코드','비중','현재가','한계리스크','절대리스크','상대리스크비중','MVO'])
mp31 = {'날짜':date_col,
      '종목명':종목명,
      '종목코드':종목코드,
      '비중':list((optsx * Wstk[2]).round(3)),
      '현재가':현재가,
      '한계리스크':MRC.round(3),
      '절대리스크':ARC.round(3),
      '상대리스크비중':PRC.round(3),
      'MVO':(front * 100).round(3)}
df31 = pd.DataFrame(mp31,columns = ['날짜','종목명','종목코드','비중','현재가','한계리스크','절대리스크','상대리스크비중','MVO'])

In [0]:
# 채권ETF 배분
raw_data2 = pd.read_csv('bond_etf.csv',index_col=['날짜'], parse_dates=['날짜'])
종목명2 = raw_data2.columns
print('\n >> 종목명 :', 종목명2)
noa2 = len(종목명2) # 자산수

symbol_data2 = pd.read_csv('symbol_cd2.csv',index_col=None,dtype={'종목코드':str})
종목코드2 = symbol_data2['종목코드']
print('\n >> 종목코드 :', 종목코드2)

work_data2 = raw_data2.iloc[-공분산_산출기간:]
현재가2 = list(raw_data2.iloc[-1])
print('\n >> 현재가 :', 현재가2)

로그수익률2 = np.log(work_data2 / work_data2.shift(1))
print(로그수익률2.head())

R2 = 로그수익률2.mean() * 252 # 연수익률
print(R2)

C2 = 로그수익률2.cov() * 252  # 공분산, 표준편차가 아닌 분산이다
print(C2)  

# 제약조건
cons = ({'type': 'eq', 'fun': lambda x: np.sum(x) - 1},
        {'type': 'ineq', 'fun': lambda x: (x >= 0.01).sum() - 2},   #매수 종목이 3 종목 이상되도록 조건 설정 
        {'type': 'ineq', 'fun': lambda x: (x >= 0.01).sum() - 1})

# 범위값
bnds = tuple((0, 1) for x in range(noa2))

opts = sco.minimize(min_func_sharpe, noa2 * [1. / noa2, ], (R2, C2), method='SLSQP', bounds=bnds, constraints=cons)
front2 = opts['x']  # 샤프지수 최대 포트폴리오 결과값

# 제약조건
cons2 = ({'type': 'eq', 'fun': lambda x: np.sum(x) - 1.0})

opts2 = sco.minimize(risk_budgeting_objective, noa2 * [1. / noa2, ], (R2, C2, front2), method='SLSQP', bounds=bnds,
                         constraints=cons2)
optsx2 = (opts2['x'] * 100).round(3)  # 샤프지수 최대 포트폴리오 결과값
print('\n >> 비중 :', optsx2)

rc_array2 = risk_contribution(opts2['x'],R2,C2)
MRC2 = (rc_array2[0] * 100).round(3)
ARC2 = (rc_array2[1] * 100).round(3)
PRC2 = (rc_array2[2] * 100).round(3)

stat = statistics(opts2['x'],R2,C2).round(3)  # 샤프지수 최대 포트폴리오의 수익률, 표준편차, 샤프지수
print('\n>> 수익률 = ', stat[0],', 표준편차 = ', stat[1], ', 샤프지수 = ', stat[2])
print('\n>> 검증 >>')
print('mvo=', (front2 * 100).round(3))
print('PRC=', PRC2.round(3))


# 투자자성향별 MP 비중 조정 후 저장

In [0]:
date_col2 = [current_dt for i in range(noa2)] # 날짜 칼럼 생성

mp12 = {'날짜':date_col2,
      '종목명':종목명2,
      '종목코드':종목코드2,
      '비중':list((optsx2 * (1 - Wstk[0])).round(3)),
      '현재가':현재가2,
      '한계리스크':MRC2.round(3),
      '절대리스크':ARC2.round(3),
      '상대리스크비중':PRC2.round(3),
      'MVO':(front2 * 100).round(3)}
df12 = pd.DataFrame(mp12,columns = ['날짜','종목명','종목코드','비중','현재가','한계리스크','절대리스크','상대리스크비중','MVO'])
df11 = pd.concat([df11,df12],axis=0,ignore_index=True)
print(df11)
df11.to_csv('RB_model_portfolio_'+current_dt+'_1.csv', index=None)

In [0]:
mp22 = {'날짜':date_col2,
      '종목명':종목명2,
      '종목코드':종목코드2,
      '비중':list((optsx2 * (1 - Wstk[1])).round(3)),
      '현재가':현재가2,
      '한계리스크':MRC2.round(3),
      '절대리스크':ARC2.round(3),
      '상대리스크비중':PRC2.round(3),
      'MVO':(front2 * 100).round(3)}
df22 = pd.DataFrame(mp22,columns = ['날짜','종목명','종목코드','비중','현재가','한계리스크','절대리스크','상대리스크비중','MVO'])
df21 = pd.concat([df21,df22],axis=0,ignore_index=True) 
print(df21)
df21.to_csv('RB_model_portfolio_'+current_dt+'_2.csv', index=None)

In [0]:
mp32 = {'날짜':date_col2,
      '종목명':종목명2,
      '종목코드':종목코드2,
      '비중':list((optsx2 * (1 - Wstk[2])).round(3)),
      '현재가':현재가2,
      '한계리스크':MRC2.round(3),
      '절대리스크':ARC2.round(3),
      '상대리스크비중':PRC2.round(3),
      'MVO':(front2 * 100).round(3)}
df32 = pd.DataFrame(mp32,columns = ['날짜','종목명','종목코드','비중','현재가','한계리스크','절대리스크','상대리스크비중','MVO'])
df31 = pd.concat([df31,df32],axis=0,ignore_index=True) 
print(df31)
df31.to_csv('RB_model_portfolio_'+current_dt+'_3.csv', index=None)